In [33]:
pip install arxiv

Note: you may need to restart the kernel to use updated packages.


In [34]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [35]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [36]:
wiki.name

'wikipedia'

In [37]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [38]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.langchain.com/langsmith")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(documents, OllamaEmbeddings())
retriever = vectorstore.as_retriever()
retriever

ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11c5ef210>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "langsmith_retriever", "Retrieve documents from LangSmith")
retriever_tool.name

NameError: name 'retriever' is not defined

In [ ]:
## Arxiv Tool
from langchain_community.tools.arxiv import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_api = ArxivAPIWrapper()
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api)
arxiv_tool.name

'arxiv'

In [60]:
tools = [
    wiki,
    arxiv_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/sreekanthpogula/Learnings/deploy-llm-with-langchain-fastapi-starter-kit/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000))]

In [61]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

In [68]:
### Agent Setup
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate

agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

In [41]:
from langchain import hub
# Pulling a pre-defined prompt template from LangChain Hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [70]:
## Agent Tools
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(
    tools,
    llm,
    prompt,
)

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AttributeError: 'list' object has no attribute 'bind'

In [71]:
agent_executor.invoke({
    "input": "What is the latest research on quantum computing?"
})

NameError: name 'agent_executor' is not defined